In [ ]:
import pandas as pd
import os

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import random
import pandas as pd
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
if os.name == 'nt' :
    font_family = "Malgun Gothic"
else:
    font_family = "AppleGothic"
    
sns.set(font=font_family, rc ={"axes.unicode_minus" : False})

## 파일 한번에 불러오기

In [ ]:
os.listdir('C:/Users/user/Desktop/생활인구 분석/data') # 폴더안에 데이터 확인

### 1. 파일리스트 저장

In [ ]:
forders= os.listdir('C:/Users/user/Desktop/생활인구 분석/data')

### 2. 현재 작업위치(working directory) 이동

In [ ]:
cd C:/Users/user/Desktop/생활인구 분석/data

### 3. 데이터 프레임 생성하고 모든 파일 합치기

In [ ]:
# 새로운 데이터 프레임 생성
df_all = pd.DataFrame()

for files in forders:
    df= pd.read_csv(files)
    df_all= pd.concat([df_all, df])

## 인덱스 생성이 안되고 하나씩 밀려 마지막 컬럼 데이터가 na값을 떠서 전처리 진행

In [ ]:
df_all = df_all.reset_index()
columns=['index', '기준일ID', '시간대구분', '행정동코드','총생활인구수','남자0세부터9세생활인구수','남자10세부터14세생활인구수',
         '남자15세부터19세생활인구수','남자20세부터24세생활인구수','남자25세부터29세생활인구수','남자30세부터34세생활인구수',
         '남자35세부터39세생활인구수','남자40세부터44세생활인구수','남자45세부터49세생활인구수','남자50세부터54세생활인구수',
         '남자55세부터59세생활인구수','남자60세부터64세생활인구수','남자65세부터69세생활인구수','남자70세이상생활인구수',
         '여자0세부터9세생활인구수', '여자10세부터14세생활인구수','여자15세부터19세생활인구수','여자20세부터24세생활인구수',
         '여자25세부터29세생활인구수','여자30세부터34세생활인구수','여자35세부터39세생활인구수','여자40세부터44세생활인구수',
         '여자45세부터49세생활인구수','여자50세부터54세생활인구수','여자55세부터59세생활인구수','여자60세부터64세생활인구수',
         '여자65세부터69세생활인구수','여자70세이상생활인구수'
        ]
df_all.columns = ['기준일ID', '시간대구분', '행정동코드','총생활인구수','남자0세부터9세생활인구수','남자10세부터14세생활인구수',
         '남자15세부터19세생활인구수','남자20세부터24세생활인구수','남자25세부터29세생활인구수','남자30세부터34세생활인구수',
         '남자35세부터39세생활인구수','남자40세부터44세생활인구수','남자45세부터49세생활인구수','남자50세부터54세생활인구수',
         '남자55세부터59세생활인구수','남자60세부터64세생활인구수','남자65세부터69세생활인구수','남자70세이상생활인구수',
         '여자0세부터9세생활인구수', '여자10세부터14세생활인구수','여자15세부터19세생활인구수','여자20세부터24세생활인구수',
         '여자25세부터29세생활인구수','여자30세부터34세생활인구수','여자35세부터39세생활인구수','여자40세부터44세생활인구수',
         '여자45세부터49세생활인구수','여자50세부터54세생활인구수','여자55세부터59세생활인구수','여자60세부터64세생활인구수',
         '여자65세부터69세생활인구수','여자70세이상생활인구수','삭제할거']
df_all = df_all.drop(['삭제할거'], axis=1)

## 동대문구의 행정동만 추출

In [ ]:
code = [
    11230710, #회기동
    11230720, # 휘경1동
    11230730, # 휘경2동
    11230705, # 청량리동
    11230536, # 용신동
    11230545, # 제기동
    11230560, # 전농1동
    11230570, # 전농2동
    11230610, # 답십리2동
    11230650, # 장안1동
    11230660, # 장안2동
    11230740, # 이문1동
    11230750, # 이문2동
    11230600  # 답십리1동
]
df_all = df_all[df_all['행정동코드'].isin(code)]

## 기준일ID 컬럼을 date형식으로 변환

In [ ]:
# 년, 월 컬럼 생성
def create_dates(df_temp):
    df_temp.insert(0, '날짜', pd.to_datetime(df_temp['기준일ID'], format='%Y%m%d'))
    df_temp.insert(1, '년', df_temp['날짜'].dt.year)
    df_temp.insert(2, '월', df_temp['날짜'].dt.month)

    # 주말 여부 컬럼 생성
    df_temp.insert(3, "주말여부", df_temp['날짜'].dt.dayofweek > 4)
    df_temp['주말여부'] = df_temp['주말여부'].apply(lambda x: '주말' if x==True else '평일')

    # 기준일ID컬럼 drop
    df_temp.drop(columns=['기준일ID'], inplace=True)
    return df_temp

create_dates(df_all)

## 총 생활인구수 int 형변환

In [ ]:
df_all = df_all.astype({'총생활인구수' : 'int'})

## 동대문구 월 평균 생활인구

### 그래프 만드는 함수

In [ ]:
def bar_Graph(width, values, count, g_x, g_y, g_title) : 
    arr = np.arange(count)
    color = []
    for i in range(count) : # 바 그래프 랜덤 색상 만들기
        if (color is None)  :
            color = ["#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
        color.append("#" + ''.join([random.choice('0123456789ABCDEF') for n in range(6)]))
    
    
    plt.figure(figsize=(10, 8))

    plt.bar(arr, 
            values, 
            color=color, # 색상 
            alpha = 0.5, # 투명도
            edgecolor = 'black', # 모서리 색상
            linewidth = 2.5 # 테두리 두께
           )
    
    plt.xticks(arr, width, fontsize=12)
    
    plt.title(g_title, fontsize=12) # 타이틀
    plt.xlabel(g_x, fontsize=12) # x 축 라벨링
    plt.ylabel(g_y, fontsize=12) # y 축 라벨링
    
    add_value_label(width, np.round(values, 3))
    
    plt.show()

### 그래프의 가운데 값 표시하는 함수

In [ ]:
# 그래프 가운데 값 표시
def add_value_label(x_list,y_list):
    for i in range(1, len(x_list)+1):
        plt.text(i-1,y_list[i-1],y_list[i-1], 
                 fontsize=15,
                 color="blue", # 글자 색상
                 ha="center")

In [ ]:
bar_Graph(['11월','12월','1월', '2월','3월','4월','5월','6월','7월','8월','9월','10월'], 
          
          [df_all.loc[df_all['월'] == 11, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 11, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 12, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 12, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 1, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 1, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 2, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 2, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 3, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 3, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 4, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 4, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 5, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 5, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 6, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 6, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 7, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 7, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 8, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 8, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 9, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 9, '총생활인구수'].count(),
           df_all.loc[df_all['월'] == 10, '총생활인구수'].sum() // df_all.loc[df_all['월'] == 10, '총생활인구수'].count()
          ], 12, '월', '생활인구(명)', '월별 평균 생활인구') # 가로이름, 값, 갯수, x라벨, y라벨, 제목

## 평일 주말 구분 평균 생활인구

In [ ]:
bar_Graph(['평일', '주말'], 
          
          [df_all.loc[df_all['주말여부'] == '평일', '총생활인구수'].sum() // df_all.loc[df_all['주말여부'] == '평일', '총생활인구수'].count(),
           df_all.loc[df_all['주말여부'] == '주말', '총생활인구수'].sum() // df_all.loc[df_all['주말여부'] == '주말', '총생활인구수'].count()
          ], 2, ' ', '생활인구(명)', '평균 생활인구') # 가로이름, 값, 갯수, x라벨, y라벨, 제목

## 성별 평균 생활인구

In [ ]:
df_all['남성합'] = df_all[['남자0세부터9세생활인구수','남자10세부터14세생활인구수',
         '남자15세부터19세생활인구수','남자20세부터24세생활인구수','남자25세부터29세생활인구수','남자30세부터34세생활인구수',
         '남자35세부터39세생활인구수','남자40세부터44세생활인구수','남자45세부터49세생활인구수','남자50세부터54세생활인구수',
         '남자55세부터59세생활인구수','남자60세부터64세생활인구수','남자65세부터69세생활인구수','남자70세이상생활인구수']].sum(axis=1)

df_all['여성합'] = df_all[['여자0세부터9세생활인구수', '여자10세부터14세생활인구수','여자15세부터19세생활인구수','여자20세부터24세생활인구수',
         '여자25세부터29세생활인구수','여자30세부터34세생활인구수','여자35세부터39세생활인구수','여자40세부터44세생활인구수',
         '여자45세부터49세생활인구수','여자50세부터54세생활인구수','여자55세부터59세생활인구수','여자60세부터64세생활인구수',
         '여자65세부터69세생활인구수','여자70세이상생활인구수']].sum(axis=1

In [ ]:
bar_Graph(['남', '여'], 
          
          [df_all['남성합'].sum() // df_all['남성합'].count(),
           df_all['여성합'].sum() // df_all['여성합'].count()
          ], 2, '성별', '생활인구(명)', '성별 평균 생활인구') # 가로이름, 값, 갯수, x라벨, y라벨, 제목

## 코드번호 -> 행정동명으로 변경

In [ ]:
df_all = df_all.replace([11230710, #회기동
        11230720, # 휘경1동
        11230730, # 휘경2동
        11230705, # 청량리동
        11230536, # 용신동
        11230545, # 제기동
        11230560, # 전농1동
        11230570, # 전농2동
        11230610, # 답십리2동
        11230650, # 장안1동
        11230660, # 장안2동
        11230740, # 이문1동
        11230750, # 이문2동
        11230600], # 답십리1동
               ['회기동', '휘경1동', '휘경2동', '청량리동', '용신동', '제기동', '전농1동', '전농2동', '답십리2동', '장안1동', '장안2동', '이문1동', '이문2동', '답십리1동']
)

## 행정동별 평균 생활인구

In [ ]:
pop_mean = df_all.groupby('행정동코드').총생활인구수.sum() // df_all.groupby('행정동코드').총생활인구수.count()
pop_mean = pop_mean.sort_values()

label = ['이문2동', '전농2동', '휘경1동', '청량리동', '회기동', '휘경2동', '답십리2동', '답십리1동', '이문1동', '장안2동', '전농1동', 
         '제기동', '장안1동', '용신동']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.barh(index, pop_mean)

plt.title('행정동별 평균 생활인구', fontsize=18)

plt.ylabel('행정동', fontsize=15)

plt.xlabel('생활인구(명)', fontsize=15)

plt.yticks(index, label, fontsize=13, rotation=0)

plt.show()

## 시간대별 동대문구 평균 생활인구

In [ ]:
def ddm_time_pop_mean() :
    pop_time_mean = df_all.groupby('시간대구분').총생활인구수.sum() // df_all.groupby('시간대구분').총생활인구수.count()
    label = []
    for i in range(len(pop_time_mean)) :
        label.append(str(i)+'시')

    index = np.arange(len(label))

    plt.figure(figsize=(15, 10))

    plt.bar(index, pop_time_mean)

    plt.title('시간대별 평균 생활인구', fontsize=20)

    plt.xlabel('시간(시)', fontsize=18)

    plt.ylabel('생활인구(명)', fontsize=18)

    plt.xticks(index, label, fontsize=15)

    plt.show()

ddm_time_pop_mean()

## 지역별 시간대 평균 생활인구

In [ ]:
def time_pop_mean_gp(region) :
    pop_time_mean2 = df_all.groupby(['행정동코드', '시간대구분']).총생활인구수.sum() // df_all.groupby(['행정동코드', '시간대구분']).총생활인구수.count()
    
    label = []
    for i in range(len(pop_time_mean2[region])) :
        label.append(str(i)+'시')

    index = np.arange(len(label))
    
    plt.figure(figsize=(15, 10))

    plt.bar(index, pop_time_mean2[region])

    plt.title('시간대별 평균 생활인구' + '(' + region + ')', fontsize=20)

    plt.xlabel('시간(시)', fontsize=18)

    plt.ylabel('생활인구(명)', fontsize=18)

    plt.xticks(index, label, fontsize=15)

    plt.show()

time_pop_mean_gp('행정동 이름 입력')

## 지역별 월별 생활인구

In [ ]:
def month_pop_mean_gp(region) :
    pop_month_mean = df_all.groupby(['행정동코드', '월']).총생활인구수.sum() // df_all.groupby(['행정동코드', '월']).총생활인구수.count()
    
    label = []
    for i in range(len(pop_month_mean[region])) :
        label.append(str(i+1)+'월')

    index = np.arange(len(label))
    
    plt.figure(figsize=(15, 10))

    plt.bar(index, pop_month_mean[region])

    plt.title('월별 평균 생활인구' + '(' + region + ')', fontsize=20)

    plt.xlabel('월', fontsize=15)

    plt.ylabel('생활인구', fontsize=15)
    
    plt.xticks(index, label, fontsize=13, rotation=0)

    plt.show()
    
month_pop_mean_gp('행정동 이름 입력')

## 연령별 평균 생활인구
### 5살 주기를 10살 주기로 변경 코드

In [ ]:
def combine_ages(df_temp):
    cols = [
    '남자0세부터9세생활인구수', '여자0세부터9세생활인구수', 
    '남자10세부터14세생활인구수', '남자15세부터19세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', 
    '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '여자20세부터24세생활인구수', '여자25세부터29세생활인구수',
    '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수', 
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', 
    '남자50세부터54세생활인구수', '남자55세부터59세생활인구수', '여자50세부터54세생활인구수', '여자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', 
    '남자70세이상생활인구수', '여자70세이상생활인구수'
    ]
    df_temp['10세 이하'] = df_temp.남자0세부터9세생활인구수 + df_temp.여자0세부터9세생활인구수
    df_temp['10대'] = df_temp.남자10세부터14세생활인구수 + df_temp.남자15세부터19세생활인구수 + df_temp.여자10세부터14세생활인구수 + df_temp.여자15세부터19세생활인구수
    df_temp['20대'] = df_temp.남자20세부터24세생활인구수 + df_temp.남자25세부터29세생활인구수 + df_temp.여자20세부터24세생활인구수 + df_temp.여자25세부터29세생활인구수
    df_temp['30대'] = df_temp.남자30세부터34세생활인구수 + df_temp.남자35세부터39세생활인구수 + df_temp.여자30세부터34세생활인구수 + df_temp.여자35세부터39세생활인구수
    df_temp['40대'] = df_temp.남자40세부터44세생활인구수 + df_temp.남자45세부터49세생활인구수 + df_temp.여자40세부터44세생활인구수 + df_temp.여자45세부터49세생활인구수
    df_temp['50대'] = df_temp.남자50세부터54세생활인구수 + df_temp.남자55세부터59세생활인구수 + df_temp.여자50세부터54세생활인구수 + df_temp.여자55세부터59세생활인구수
    df_temp['60대'] = df_temp.남자60세부터64세생활인구수 + df_temp.남자65세부터69세생활인구수 + df_temp.여자60세부터64세생활인구수 + df_temp.여자65세부터69세생활인구수
    df_temp['70세 이상'] = df_temp.남자70세이상생활인구수 + df_temp.여자70세이상생활인구수
    df_temp.drop(columns=cols, inplace=True)
    df_temp = df_temp.groupby(['날짜', '월', '주말여부', '행정동코드']).sum()
    df_temp = df_temp.groupby(['월', '주말여부', '행정동코드']).mean()


    return df_temp

## 작년 수능 생활인구 2021년 11월 18일 목요일

In [ ]:
pop_mean = df_all[df_all['날짜'] == '2021-11-18'].groupby('행정동코드').총생활인구수.sum()
pop_mean = pop_mean.sort_values()

label = ['이문2동', '휘경1동', '전농2동', '청량리동', '회기동', '휘경2동', '답십리2동', '답십리1동', '이문1동', '장안2동', '전농1동',
        '제기동', '장안1동', '용신동']
    
index = np.arange(len(label))
index

plt.figure(figsize=(15, 10))

plt.barh(index, pop_mean)

plt.title('작년 수능날 동대문구 생활인구', fontsize=18)

plt.ylabel('행정동', fontsize=15)

plt.xlabel('생활인구(명)', fontsize=15)

plt.yticks(index, label, fontsize=13, rotation=0)

plt.show()

## 작년 수능날 시간대별 생활인구

In [ ]:
def sssss(region) :
    pop_time_mean2 = df_all[df_all['날짜'] == '2021-11-18'].groupby(['행정동코드', '시간대구분']).총생활인구수.sum()
    label = []
    for i in range(len(pop_time_mean2[region])) :
        label.append(str(i)+'시')

    index = np.arange(len(label))
    
    plt.figure(figsize=(15, 10))

    plt.bar(index, pop_time_mean2[region])

    plt.title('작년 수능날 시간대별 평균 생활인구' + '(' + region + ')', fontsize=20)

    plt.xlabel('시간(시)', fontsize=18)

    plt.ylabel('생활인구(명)', fontsize=18)

    plt.xticks(index, label, fontsize=15)

    plt.show()

sssss('행정동 이름 입력')